<a href="https://colab.research.google.com/github/nthanhkhang/Natural-Language-Processing/blob/main/POS_tagging_by_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [1]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [2]:
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\nguyn\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\nguyn\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [3]:
sent = nltk_data[0]
for i in range(len(sent)):
    print(sent[i][0],sent[i][1])

Pierre NOUN
Vinken NOUN
, .
61 NUM
years NOUN
old ADJ
, .
will VERB
join VERB
the DET
board NOUN
as ADP
a DET
nonexecutive ADJ
director NOUN
Nov. NOUN
29 NUM
. .


# Split data into training and validation set in the ratio 80:20

In [4]:
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

# Create list of train and test tagged words

In [5]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


# Check some of the tagged words.

In [6]:
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [7]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'PRT', 'PRON', '.', 'X', 'ADP', 'ADV', 'NOUN', 'VERB', 'ADJ', 'NUM', 'CONJ', 'DET'}


# Compute Emission Probability
# P(tj|ti) ->log
# P(wi|tj) ->log

In [8]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [9]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [10]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[1.17416831e-03 1.76125243e-02 4.50097844e-02 1.21330721e-02
  1.95694715e-02 9.39334650e-03 2.50489235e-01 4.01174158e-01
  8.29745606e-02 5.67514673e-02 2.34833662e-03 1.01369865e-01]
 [1.41230067e-02 6.83371304e-03 4.19134386e-02 8.83826911e-02
  2.23234631e-02 3.69020514e-02 2.12756261e-01 4.84738052e-01
  7.06150308e-02 6.83371304e-03 5.01138950e-03 9.56719834e-03]
 [2.78940029e-03 6.87694475e-02 9.23720598e-02 2.56410260e-02
  9.29084867e-02 5.25694676e-02 2.18538776e-01 8.96899477e-02
  4.61323895e-02 7.82104954e-02 6.00793920e-02 1.72191828e-01]
 [1.85085520e-01 5.41995019e-02 1.60868734e-01 7.57255405e-02
  1.42225638e-01 2.57543717e-02 6.16951771e-02 2.06419379e-01
  1.76821072e-02 3.07514891e-03 1.03786280e-02 5.68902567e-02]
 [1.26550242e-03 6.96026310e-02 3.87243740e-02 3.45482156e-02
  1.69577319e-02 1.45532778e-02 3.23588967e-01 8.47886596e-03
  1.07061505e-01 6.32751212e-02 1.01240189e-03 3.20931405e-01]
 [1.47401085e-02 1.20248254e-02 1.39255241e-01 2.28859577e-02
  1

In [11]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,PRT,PRON,.,X,ADP,ADV,NOUN,VERB,ADJ,NUM,CONJ,DET
PRT,0.001174,0.017613,0.045010,0.012133,0.019569,0.009393,0.250489,0.401174,0.082975,0.056751,0.002348,0.101370
PRON,0.014123,0.006834,0.041913,0.088383,0.022323,0.036902,0.212756,0.484738,0.070615,0.006834,0.005011,0.009567
.,0.002789,0.068769,0.092372,0.025641,0.092908,0.052569,0.218539,0.089690,0.046132,0.078210,0.060079,0.172192
X,0.185086,0.054200,0.160869,0.075726,0.142226,0.025754,0.061695,0.206419,0.017682,0.003075,0.010379,0.056890
ADP,0.001266,0.069603,0.038724,0.034548,0.016958,0.014553,0.323589,0.008479,0.107062,0.063275,0.001012,0.320931
ADV,0.014740,0.012025,0.139255,0.022886,0.119472,0.081458,0.032196,0.339022,0.130721,0.029868,0.006982,0.071373
NOUN,0.043935,0.004659,0.240094,0.028825,0.176827,0.016895,0.262344,0.149134,0.012584,0.009144,0.042454,0.013106
VERB,0.030663,0.035543,0.034807,0.215930,0.092357,0.083886,0.110589,0.167956,0.066390,0.022836,0.005433,0.133610
ADJ,0.011456,0.000194,0.066019,0.020971,0.080583,0.005243,0.696893,0.011456,0.063301,0.021748,0.016893,0.005243
NUM,0.026062,0.001428,0.119243,0.202428,0.037487,0.003570,0.351660,0.020707,0.035345,0.184220,0.014281,0.003570


# Viterbi Algorithm

In [12]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [13]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [14]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
# accuracy
# check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  68.32021403312683
Viterbi Algorithm Accuracy:  93.77990430622009


In [15]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [16]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

# Test accuracy on subset of test data 


In [17]:
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  74.15143156051636
Viterbi Algorithm Accuracy:  97.1291866028708


In [18]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)
#Will and Marry are tagged as NUM as they are unknown words for Viterbi Algorithm 

[('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('Will', 'PRT'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'PRT')]
